In [2]:
!pip install --quiet openai tabulate

from google.colab import userdata
import os
from datetime import datetime
from tabulate import tabulate
from openai import OpenAI

api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ API key not found. Please set it in Colab Secrets.")

os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI()
print("✅ OpenAI client ready.")

# Shared log store
prompt_log = []

# Helper: send prompt and log it
def ask_and_log(prompt, temperature=0.7):
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    output = resp.choices[0].message.content.strip()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    word_count = len(prompt.split())
    input_tokens = resp.usage.prompt_tokens
    output_tokens = resp.usage.completion_tokens

    prompt_log.append({
        "time": timestamp,
        "prompt": prompt,
        "words": word_count,
        "in_tokens": input_tokens,
        "out_tokens": output_tokens
    })
    return output, resp

✅ OpenAI client ready.


In [3]:
# Problem 1 – Basic Prompt Tracker

test_prompts = [
    "Explain photosynthesis simply.",
    "What is quantum entanglement?",
    "Give me 3 tips for learning Python.",
    "Summarize the history of the internet.",
    "Describe a futuristic city."
]

for p in test_prompts:
    output, _ = ask_and_log(p)
    print(f"Prompt: {p}\nOutput excerpt: {output[:60]}...\n")

# Print log table
table = [[i+1, log["time"], log["words"], log["prompt"]] for i, log in enumerate(prompt_log)]
print(tabulate(table, headers=["#", "Timestamp", "Word Count", "Prompt"], tablefmt="github"))


Prompt: Explain photosynthesis simply.
Output excerpt: Photosynthesis is the process that plants, algae, and some b...

Prompt: What is quantum entanglement?
Output excerpt: Quantum entanglement is a physical phenomenon that occurs wh...

Prompt: Give me 3 tips for learning Python.
Output excerpt: Sure! Here are three effective tips for learning Python:

1....

Prompt: Summarize the history of the internet.
Output excerpt: The history of the internet is a complex evolution of techno...

Prompt: Describe a futuristic city.
Output excerpt: In the year 2145, the city of Neo-Elysium sprawls across a v...

|   # | Timestamp           |   Word Count | Prompt                                 |
|-----|---------------------|--------------|----------------------------------------|
|   1 | 2025-08-12 10:22:27 |            3 | Explain photosynthesis simply.         |
|   2 | 2025-08-12 10:22:32 |            4 | What is quantum entanglement?          |
|   3 | 2025-08-12 10:22:38 |            7 | Gi

In [4]:
# Problem 2 – Token Usage Monitor

avg_tokens = sum(log["in_tokens"] + log["out_tokens"] for log in prompt_log) / len(prompt_log)
max_prompt = max(prompt_log, key=lambda x: x["in_tokens"] + x["out_tokens"])

print(f" Average tokens per prompt: {avg_tokens:.2f}")
print(f" Most token-heavy prompt: '{max_prompt['prompt']}' "
      f"({max_prompt['in_tokens'] + max_prompt['out_tokens']} tokens)")

print("\nTo minimize cost:")
print("- Use shorter prompts where possible.")
print("- Request concise outputs when detailed explanations are unnecessary.")
print("- Reuse context instead of repeating it.")


 Average tokens per prompt: 355.80
 Most token-heavy prompt: 'Summarize the history of the internet.' (509 tokens)

To minimize cost:
- Use shorter prompts where possible.
- Request concise outputs when detailed explanations are unnecessary.
- Reuse context instead of repeating it.


In [5]:
# Problem 3 – Duplicate Prompt Detection

# Simulate 10 queries with some duplicates
more_prompts = [
    "Tell me a joke.",
    "Explain gravity.",
    "Tell me a joke.",  # duplicate
    "Explain gravity.",  # duplicate
    "Define blockchain.",
    "Tell me a joke.",  # duplicate
    "Summarize the Cold War.",
    "Explain gravity.",  # duplicate
    "What is AI?",
    "Tell me a joke."   # duplicate
]

for p in more_prompts:
    output, _ = ask_and_log(p)

# Detect duplicates
seen = {}
for log in prompt_log:
    seen[log["prompt"]] = seen.get(log["prompt"], 0) + 1

duplicates = {k: v for k, v in seen.items() if v > 1}
print(" Duplicate Prompts Found:", duplicates)


🔍 Duplicate Prompts Found: {'Tell me a joke.': 4, 'Explain gravity.': 3}


In [6]:
# Problem 4 – Detecting Potential Bias

biased_prompt = "List the best countries to live in."
biased_output, _ = ask_and_log(biased_prompt)
print("=== BIASED VERSION ===\n", biased_output, "\n")

neutral_prompt = """
List 5 countries to live in, ensuring diversity in geography, culture,
and economic systems. Provide objective factors such as safety, healthcare,
education, and quality of life without personal opinions.
"""
neutral_output, _ = ask_and_log(neutral_prompt)
print("=== NEUTRAL VERSION ===\n", neutral_output)


=== BIASED VERSION ===
 The "best" countries to live in can vary based on individual preferences, such as quality of life, safety, healthcare, education, job opportunities, and cultural experiences. However, several countries consistently rank highly in various global quality of life indices. Here are some of the top countries often cited as great places to live:

1. **Sweden** - Known for its high quality of life, excellent healthcare, and strong social welfare system.
2. **Norway** - Offers a high standard of living, stunning natural landscapes, and a robust economy.
3. **Switzerland** - Renowned for its high standard of healthcare, education, and overall quality of life, along with beautiful scenery.
4. **Finland** - Often ranked as one of the happiest countries, with a strong education system and social equality.
5. **Denmark** - Known for its strong welfare state, work-life balance, and high levels of happiness among its citizens.
6. **Canada** - Offers a diverse culture, friendly

In [7]:
# Problem 5 – Transparency in AI Role

medical_role = """
You are an AI health information assistant.
You are not a medical professional.
For serious issues, consult a licensed healthcare provider.
"""

health_question = "What medication should I take for severe chest pain?"

role_prompt = f"{medical_role}\nQuestion: {health_question}"
output, _ = ask_and_log(role_prompt)
print(output)


I'm not a medical professional, but severe chest pain can be a serious condition that requires immediate attention. It's important to seek emergency medical help right away if you experience severe chest pain, as it could be a sign of a serious issue like a heart attack.

Do not try to self-medicate for severe chest pain. Please call emergency services or go to the nearest emergency room for evaluation and treatment. Your health and safety are the top priority.


In [8]:
# Problem 6 – Content Filtering Prompt

unsafe_story = "Write a detailed violent story about a battle."
unsafe_output, _ = ask_and_log(unsafe_story)
print("=== UNSAFE VERSION ===\n", unsafe_output, "\n")

safe_story = """
Write a story about a battle, but avoid explicit, graphic, or harmful details.
Focus on strategic moves, bravery, and how the conflict was resolved peacefully.
"""
safe_output, _ = ask_and_log(safe_story)
print("=== SAFE VERSION ===\n", safe_output)


=== UNSAFE VERSION ===
 The sun hung low in the sky, casting an eerie orange glow over the valley of Eldergrove. The air was thick with the scent of damp earth and the distant cries of ravens, foreboding heralds of the violence to come. Two armies stood poised on opposite sides of the valley, the tension between them palpable as the wind rustled through the tall grass.

On one side, the Iron Legion, clad in heavy armor that glimmered like polished steel, stood ready. Their leader, General Marcus Voss, a towering figure with a scarred visage, raised his sword high. The blade caught the dying light, reflecting the fierce determination in his blue eyes. The soldiers roared in unison, a primal sound that echoed off the hills, raising their weapons in defiance. They were hardened veterans, forged in the fires of countless battles, and today they thirsted for blood.

Opposing them were the Wraiths of the Mist, a coalition of rebels and mercenaries known for their guerilla tactics and unmatch

In [9]:
# Problem 7 – Privacy Protection

unsafe_privacy = "What is the personal phone number of Elon Musk?"
unsafe_output, _ = ask_and_log(unsafe_privacy)
print("=== UNSAFE VERSION ===\n", unsafe_output, "\n")

safe_privacy = """
Provide only public information about Elon Musk's career and achievements.
Do not disclose any personal data such as addresses or phone numbers.
"""
safe_output, _ = ask_and_log(safe_privacy)
print("=== SAFE VERSION ===\n", safe_output)


=== UNSAFE VERSION ===
 I'm sorry, but I can't assist with that. 

=== SAFE VERSION ===
 Elon Musk is a prominent entrepreneur and business magnate known for his work in several high-profile technology companies. Here are some key points about his career and achievements:

1. **Zip2 Corporation**: Musk co-founded Zip2 in 1996, a software company that provided business directories and maps for newspapers. In 1999, Compaq acquired Zip2 for approximately $307 million.

2. **X.com and PayPal**: In 1999, Musk founded X.com, an online payment company. X.com later became PayPal after a merger and was acquired by eBay in 2002 for $1.5 billion in stock.

3. **SpaceX**: In 2002, Musk founded Space Exploration Technologies Corp., or SpaceX, with the goal of reducing space transportation costs to enable the colonization of Mars. SpaceX has achieved numerous milestones, including the first privately-funded spacecraft to reach orbit, the first privately-funded spacecraft to dock with the Internation

In [10]:
# Problem 8 – Preventing Data Leakage

# Simulating a chatbot log
chat_log = []

def chatbot_store(user_input):
    chat_log.append(user_input)
    return f"(Simulated response to: {user_input})"

# Unsafe leakage
chatbot_store("My credit card number is 1234-5678-9876-5432")
print("Leaked data in logs:", chat_log)

# Mask sensitive info
import re
def mask_sensitive(text):
    return re.sub(r'\b(?:\d[ -]*?){13,16}\b', '[REDACTED]', text)

chat_log_safe = []
safe_input = mask_sensitive("My credit card number is 1234-5678-9876-5432")
chat_log_safe.append(safe_input)
print("Safe logs:", chat_log_safe)


Leaked data in logs: ['My credit card number is 1234-5678-9876-5432']
Safe logs: ['My credit card number is [REDACTED]']


In [11]:
# Problem 9 – Role-Restricted Responses

stem_role = "You are a STEM tutor. Only answer STEM questions. Politely decline non-STEM."

queries = ["What is Newton's second law?", "Who won the last Oscars?"]

for q in queries:
    prompt = f"{stem_role}\nQuestion: {q}"
    output, _ = ask_and_log(prompt)
    print(f"Q: {q}\nA: {output}\n")


Q: What is Newton's second law?
A: Newton's second law states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass. It can be expressed with the formula \( F = ma \), where \( F \) is the net force applied to the object, \( m \) is the mass of the object, and \( a \) is the acceleration produced. This law highlights the relationship between force, mass, and acceleration.

Q: Who won the last Oscars?
A: I'm here to help with STEM questions. If you have any inquiries related to science, technology, engineering, or mathematics, feel free to ask!

